# Preparation

If you installed Kubeflow via [kfctl](https://www.kubeflow.org/docs/gke/customizing-gke/#common-customizations), you may already prepared GPU enviroment and can skip this section.

If you installed Kubeflow Pipelines via [Google Cloud AI Platform Pipelines UI](https://console.cloud.google.com/ai-platform/pipelines/) or [Standalone manifest](https://github.com/kubeflow/pipelines/tree/master/manifests/kustomize), please follow following steps to setup GPU enviroment.

## Add GPU nodes to your cluster

To see which accelerators are available in each zone, run the following command or check the [document](https://cloud.google.com/compute/docs/gpus#gpus-list)

```
gcloud compute accelerator-types list
```

You may also check or edit the GCP's **GPU Quota** to make sure you still have GPU quota in the region.

To well saving the costs, it's possible you create a zero-sized node pool for GPU and enable the autoscaling.

Here is an example to create a P100 GPU node pool for a cluster.

```shell
# You may customize these parameters.
export GPU_POOL_NAME=p100pool
export CLUSTER_NAME=existingClusterName
export CLUSTER_ZONE=us-west1-a
export GPU_TYPE=nvidia-tesla-p100
export GPU_COUNT=1
export MACHINE_TYPE=n1-highmem-16


# It may takes several minutes.
gcloud container node-pools create ${GPU_POOL_NAME} \
  --accelerator type=${GPU_TYPE},count=${GPU_COUNT} \
  --zone ${CLUSTER_ZONE} --cluster ${CLUSTER_NAME} \
  --num-nodes=0 --machine-type=${MACHINE_TYPE} --min-nodes=0 --max-nodes=5 --enable-autoscaling \
  --scopes=cloud-platform
```

Here in this sample, we specified **--scopes=cloud-platform**. More info is [here](https://cloud.google.com/sdk/gcloud/reference/container/node-pools/create#--scopes). It will allow the job in the node pool to use GCE Default Service Account to access GCP APIs (e.x. GCS etc.). You also use [Workload Identity](https://cloud.google.com/kubernetes-engine/docs/how-to/workload-identity) or [Application Default Credential](https://cloud.google.com/docs/authentication/production) to replace **--scopes=cloud-platform**.

## Install device driver to the cluster

After adding GPU nodes to your cluster, you need to install NVIDIA’s device drivers to the nodes. Google provides a DaemonSet that automatically installs the drivers for you.

To deploy the installation DaemonSet, run the following command. It's an one-off work.

```shell
kubectl apply -f https://raw.githubusercontent.com/GoogleCloudPlatform/container-engine-accelerators/master/nvidia-driver-installer/cos/daemonset-preloaded.yaml
```

# Consume GPU via Kubeflow Pipelines SDK

Here is a [document](https://www.kubeflow.org/docs/gke/pipelines/enable-gpu-and-tpu/).

Following is a sample quick smoking test.


In [3]:
import kfp
from kfp import dsl

def gpu_smoking_check_op():
    return dsl.ContainerOp(
        name='check',
        image='tensorflow/tensorflow:latest-gpu',
        command=['sh', '-c'],
        arguments=['nvidia-smi']
    ).set_gpu_limit(1)

@dsl.pipeline(
    name='GPU smoking check',
    description='Smoking check whether GPU env is ready.'
)
def gpu_pipeline():
    gpu_smoking_check = gpu_smoking_check_op()

if __name__ == '__main__':
    kfp.compiler.Compiler().compile(gpu_pipeline, 'gpu_smoking_check.yaml')

You may see warning message from Kubeflow Pipeline logs saying "Insufficient nvidia.com/gpu". Please wait for few minutes.

If everything runs well, it's expected to see the results of "nvidia-smi" mentions the CUDA version, GPU type and usage etc.

> You may also notice that after the pod got finished, the new GPU node is still there. GKE autoscale algorithm will free that node if no usage for certain time. More info is [here](https://cloud.google.com/kubernetes-engine/docs/concepts/cluster-autoscaler).

# Multiple GPUs pool in one cluster

It's possible you want more then 1 type of GPU to be supported in one cluster.

- There are several types of GPUs.
- Certain regions normally just support part of the GPUs ([document](https://cloud.google.com/compute/docs/gpus#gpus-list)).

Since we can set "--num-nodes=0" for certain GPU node pool to save costs if no workload, we can create multiple node pools for different types of GPUs.

## Add additional GPU nodes to your cluster


In upper section, we added a node pool for P100. Here we add another pool for V100.

```shell
# You may customize these parameters.
export GPU_POOL_NAME=v100pool
export CLUSTER_NAME=existingClusterName
export CLUSTER_ZONE=us-west1-a
export GPU_TYPE=nvidia-tesla-v100
export GPU_COUNT=1
export MACHINE_TYPE=n1-highmem-8


# It may takes several minutes.
gcloud container node-pools create ${GPU_POOL_NAME} \
  --accelerator type=${GPU_TYPE},count=${GPU_COUNT} \
  --zone ${CLUSTER_ZONE} --cluster ${CLUSTER_NAME} \
  --num-nodes=0 --machine-type=${MACHINE_TYPE} --min-nodes=0 --max-nodes=5 --enable-autoscaling
```

## Consume certain GPU via Kubeflow Pipelines SDK

Please reference following sample which explictlly request to use certain GPU.

In [4]:
import kfp
from kfp import dsl

def gpu_p100_op():
    return dsl.ContainerOp(
        name='check_p100',
        image='tensorflow/tensorflow:latest-gpu',
        command=['sh', '-c'],
        arguments=['nvidia-smi']
    ).set_gpu_limit(1).add_node_selector_constraint('cloud.google.com/gke-accelerator', 'nvidia-tesla-p100')

def gpu_v100_op():
    return dsl.ContainerOp(
        name='check_v100',
        image='tensorflow/tensorflow:latest-gpu',
        command=['sh', '-c'],
        arguments=['nvidia-smi']
    ).set_gpu_limit(1).add_node_selector_constraint('cloud.google.com/gke-accelerator', 'nvidia-tesla-v100')

@dsl.pipeline(
    name='GPU smoking check',
    description='Smoking check whether GPU env is ready.'
)
def gpu_pipeline():
    gpu_p100 = gpu_p100_op()
    gpu_v100 = gpu_v100_op()

if __name__ == '__main__':
    kfp.compiler.Compiler().compile(gpu_pipeline, 'gpu_smoking_check.yaml')

It's expected it runs well and you will see different "nvidia-smi" logs from the two pipeline steps.

## Preemptible GPU
Preemptible GPU resource is more cheaper but it also means your task requires retries.

Please notice the following only difference is that it added **--preemptible** and **--node-taints=preemptible=true:NoSchedule** parameters.

```
export GPU_POOL_NAME=v100pool-preemptible
export CLUSTER_NAME=existingClusterName
export CLUSTER_ZONE=us-west1-a
export GPU_TYPE=nvidia-tesla-v100
export GPU_COUNT=1
export MACHINE_TYPE=n1-highmem-8

gcloud container node-pools create ${GPU_POOL_NAME} \
  --accelerator type=${GPU_TYPE},count=${GPU_COUNT} \
  --zone ${CLUSTER_ZONE} --cluster ${CLUSTER_NAME} \
  --preemptible \
  --node-taints=preemptible=true:NoSchedule \
  --num-nodes=0 --machine-type=${MACHINE_TYPE} --min-nodes=0 --max-nodes=5 --enable-autoscaling
```

In [5]:
import kfp
import kfp.gcp as gcp
from kfp import dsl

def gpu_p100_op():
    return dsl.ContainerOp(
        name='check_p100',
        image='tensorflow/tensorflow:latest-gpu',
        command=['sh', '-c'],
        arguments=['nvidia-smi']
    ).set_gpu_limit(1).add_node_selector_constraint('cloud.google.com/gke-accelerator', 'nvidia-tesla-p100')

def gpu_v100_op():
    return dsl.ContainerOp(
        name='check_v100',
        image='tensorflow/tensorflow:latest-gpu',
        command=['sh', '-c'],
        arguments=['nvidia-smi']
    ).set_gpu_limit(1).add_node_selector_constraint('cloud.google.com/gke-accelerator', 'nvidia-tesla-v100')

def gpu_v100_preemptible_op():
    v100_op = dsl.ContainerOp(
        name='check_v100_preemptible',
        image='tensorflow/tensorflow:latest-gpu',
        command=['sh', '-c'],
        arguments=['nvidia-smi'])
    v100_op.set_gpu_limit(1)
    v100_op.add_node_selector_constraint('cloud.google.com/gke-accelerator', 'nvidia-tesla-v100')
    v100_op.apply(gcp.use_preemptible_nodepool(hard_constraint=True))
    return v100_op

@dsl.pipeline(
    name='GPU smoking check',
    description='Smoking check whether GPU env is ready.'
)
def gpu_pipeline():
    gpu_p100 = gpu_p100_op()
    gpu_v100 = gpu_v100_op()
    gpu_v100_preemptible = gpu_v100_preemptible_op()

if __name__ == '__main__':
    kfp.compiler.Compiler().compile(gpu_pipeline, 'gpu_smoking_check.yaml')

# TPU
Google's TPU is awesome. It's faster and lower TOC. To consume TPU, no need to create node-pool, just call KFP SDK to use it. Here is a [doc](https://www.kubeflow.org/docs/gke/pipelines/enable-gpu-and-tpu/#configure-containerop-to-consume-tpus). Please notice that not all regions has TPU yet.

